# Advanced ML: HW 2

Данная работа - ДЗ2 по курсу Advanced Machine Learning, академия больших данных MADE.

Необходимо разработать вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК).

Background: в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:

●	построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;

●	при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;

●	а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”.


In [7]:
from tqdm import tqdm
from scipy.stats import kendalltau, spearmanr
import numpy as np

In [ ]:
# пример представления данных
# results.iloc[0].iloc[0]

{'team': {'id': 242,
  'name': 'Команда Азимова',
  'town': {'id': 21, 'name': 'Баку'}},
 'mask': None,
 'current': {'name': 'Команда Азимова', 'town': {'id': 21, 'name': 'Баку'}},
 'questionsTotal': 0,
 'synchRequest': None,
 'position': 1,
 'controversials': [],
 'flags': [],
 'teamMembers': [{'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 476,
    'name': 'Анар',
    'patronymic': 'Беюкага оглы',
    'surname': 'Азимов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 878,
    'name': 'Фариз',
    'patronymic': 'Наим оглы',
    'surname': 'Аликишибеков'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 1872,
    'name': 'Аднан',
    'patronymic': 'Фариз оглы',
    'surname': 'Ахундов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 13721,
    'name': 'Балаш',
    'patronymic': 'Алекпер оглы',
    'surname': 'Касумов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'play

Загружаем уже готовые предобработанные данные о игроках, турнирах и результатах 

Cсылка на данные - https://drive.google.com/drive/folders/1NqXu0NL60vfre9lfelMpWERu4NE71lnU?usp=sharing

В трейновую выборку попадают турниры до 2019 года, в тест - после

In [10]:
train_dataset = pd.read_csv('train_dataset.csv')
test_dataset = pd.read_csv('test_dataset.csv')

In [11]:
train_dataset.head()

,answer,player,tournament_id,qty,team_id,position
0,1,6212,4772,1,45556,1.0
1,1,6212,4772,1,45556,1.0
2,1,6212,4772,1,45556,1.0
3,1,6212,4772,1,45556,1.0
4,1,6212,4772,1,45556,1.0


## baseline модель

в качестве первого подхода попробуем статистический, расчитаем в качестве рейтинга взвешенное среднее для каждого игрока в обучающем наборе, основываясь на сложность вопроса и результативный ответ 

In [10]:
rating_stat = dict()

def statistic_rating(df):
  users = df.player.unique()
  for u in tqdm(users):
    a = df[df['player'] == u].answer.values
    q = df[df['player'] == u].qty.values
    num = sum(a*q)
    r = num / sum(q)
    rating_stat[u] = r 

In [ ]:
# let's wait a bit
statistic_rating(train_dataset)
train_dataset.head()

100%|██████████| 59101/59101 [1:11:08<00:00, 13.85it/s]


,answer,player,tournament_id,qty,rating_stat
0,1,6212,4772,1,0
1,1,6212,4772,1,0
2,1,6212,4772,1,0
3,1,6212,4772,1,0
4,1,6212,4772,1,0


In [12]:
players_df = pd.read_csv('players.csv')

In [ ]:
players_df['rating_stat'] = players_df.id.apply(lambda x: rating_stat[x] if x in rating_stat.keys() else 0)


In [21]:
# players_df.to_csv('players.csv', index=False)

In [8]:
players_df.sort_values(by='rating_stat', ascending=False).head(10)

,id,name,patronymic,surname,rating_stat
34776,36844,Павел,Константинович,Щербина,0.972222
119225,133504,София,Евгеньевна,Лебедева,0.944444
183202,202410,Валентина,NaN,Подюкова,0.916667
172905,191332,Марина,Юрьевна,Савушкина,0.902778
195518,215497,Екатерина,NaN,Горелова,0.902778
195517,215496,Наталья,NaN,Артемьева,0.902778
195516,215495,Юлия,NaN,Крюкова,0.902778
167900,186002,Инга,Андрисовна,Лоренц,0.902778
91217,103161,Надежда,Фёдоровна,Бирюкова,0.888889
154692,171845,Михаил,Владимирович,Завьялов,0.888889


получаем рейтинг лист с рассчитаными статистиками 

Теперь в качестве baseline построим линейную регрессию, которая будет обучать рейтинг-лист игроков. Будем исходить из следующих предположений:
Единицей рейтинг-листа - игрок. Результаты всей команды  относим к каждому игроку индивидуально.
При построении модели учитываем разную сложность вопросов.
Тогда линейная модель принимает на вход id игрока и какой сожности был вопрос, таргет - 0 или 1 ответ на вопрос.  

In [12]:
train_dataset['player'] = train_dataset['player'].astype('category')
train_dataset['qty'] = train_dataset['qty'].astype('category')

X_train = pd.get_dummies(train_dataset[['player', 'qty']], sparse = True)

In [14]:
# one hot id игрока и сложности qty вопроса 
X_train

,player_15,player_16,player_23,player_31,player_35,player_38,player_47,player_59,player_65,player_79,...,qty_20,qty_21,qty_22,qty_23,qty_24,qty_25,qty_26,qty_27,qty_28,qty_29
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20910735,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20910736,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20910737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20910738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y_train = train_dataset.answer

**В качестве ретинга (силы игрока) берем его соответствующий линейный коэффициент в уравнении модели регрессии**

In [16]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [18]:
rating_stat_model = {}

all_train_players = len(train_dataset.player.unique())
coef = model.coef_
columns = X_train.columns
for i in range(all_train_players):
    id_player = columns[i][7:]
    rating_stat_model[int(id_player)] = coef[i]

In [19]:
players_df['rating_model'] = players_df.id.apply(lambda x: rating_stat_model[x] if x in rating_stat_model.keys() else 0)


In [20]:
players_df.sort_values(by='rating_model', ascending=False).head(10)

,id,name,patronymic,surname,rating_stat,rating_model
34776,36844,Павел,Константинович,Щербина,0.972222,0.611356
167900,186002,Инга,Андрисовна,Лоренц,0.902778,0.576024
183202,202410,Валентина,NaN,Подюкова,0.916667,0.576024
154692,171845,Михаил,Владимирович,Завьялов,0.888889,0.548266
180948,199963,Елена,Борисовна,Бровченко,0.875000,0.548266
195517,215496,Наталья,NaN,Артемьева,0.902778,0.548266
153870,170977,Давид,Сергеевич,Кан,0.888889,0.548266
195518,215497,Екатерина,NaN,Горелова,0.902778,0.548266
195516,215495,Юлия,NaN,Крюкова,0.902778,0.548266
119225,133504,София,Евгеньевна,Лебедева,0.944444,0.548266


Получаем расчитанный по модели рейтинг-лист игроков

# Построение прогнозов результатов турнира

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; Поэтому будем предсказывать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд; Для оценки есть тестовая выборка с урнирами.

в качестве метрики качества на тестовом наборе считаем ранговые корреляции Спирмена и Кендалла между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Ранжируем команды: суммирум силы всех игроков в команде турнира, если игрок попадался ранее в обучающей выборке, иначе его рейтинг 0. 

In [24]:
# истинный рейтинг-лист тестовой выборки
rating_list = test_dataset.groupby(['tournament_id', 'team_id']).position.mean().reset_index()


In [25]:
rating_list.head(10)

,tournament_id,team_id,position
0,4957,2,5.5
1,4957,84,31.5
2,4957,312,47.0
3,4957,928,22.0
4,4957,1799,56.0
5,4957,2421,56.0
6,4957,2792,31.5
7,4957,2937,69.0
8,4957,3156,78.0
9,4957,3875,3.0


In [22]:
tournaments = test_dataset.tournament_id.unique()
len(tournaments)

173

In [23]:
TOUR_TEAM = dict()
for t in tournaments:
  teams = test_dataset[test_dataset['tournament_id'] == t].team_id.unique()
  TOUR_TEAM[t] = teams


In [29]:
def predict(tournm, type_rating):
  teams = TOUR_TEAM[tournm]
  pos_predicted = dict()
  for t in teams:
    players = test_dataset[(test_dataset.tournament_id == tournm)&(test_dataset.team_id == t)].player.unique()
    place = players_df[players_df['id'].isin(players)][type_rating].sum()
    pos_predicted[t] = place
    res_sort = sorted(pos_predicted.items(), key=lambda x: x[1], reverse=True)
  return {team_id[0]: i + 1 for i, team_id in enumerate(res_sort)}
  

In [30]:
def val_correlations(tournaments, type_rating):
    score_spearmanr = []
    score_kendalltau = []
    for tournm in tqdm(tournaments):
        teams = TOUR_TEAM[tournm]
        rating = predict(tournm, type_rating)  
        pred_pos = [rating[t] for t in teams]
        target = [rating_list[(rating_list['tournament_id'] == tournm)&(rating_list['team_id'] == t)]['position'] for t in teams]
        score_spearmanr.append(spearmanr(pred_pos, target)[0]) 
        score_kendalltau.append(kendalltau(pred_pos, target)[0])
    return np.nanmean(np.array(score_spearmanr)), np.nanmean(np.array(score_kendalltau))

In [ ]:
val_correlations(tournaments, 'rating_stat')

100%|██████████| 173/173 [04:49<00:00,  1.67s/it]


(0.5340032815458579, 0.39857878068567404)

метрики для рейтинга по 1му подходу с рассчитаными статистиками

In [61]:
val_correlations(tournaments, 'rating_model')

100%|██████████| 173/173 [08:36<00:00,  2.99s/it]


(0.7278998268005902, 0.5692668643841021)

метрики для линейной модели: корреляция Спирмена 0.727, корреляция Кендалла: 0.569

видим, что модель справляется лучше, чем статистики

## Построение ЕМ-алгоритма 

Для каждого игрока с помощью линейной модели мы можем предсказать вероятность ответа на конкретный вопрос. Про каждый вопрос известно, ответила ли на него команда или нет. Если команда не ответила - никто из игроков не смог ответить на вопрос. Если команда ответила, то мы не знаем, кто конкретно из игроков предложил правильный ответ. Но Единицей рейтинг системы является игрок. Следовательно наблюдаемая переменная - ответ на вопрос. Скрытая переменная - веротяность того, что на вопрос ответил данный конкретный игрок.

На первой итерации оставляем предположение из baseline, добавляя на выход линейной модели сигмоиду для предсказания вероятности ответа на вопрос. 
Е - шаг: аналогично выше получаем вероятности ответа в тех, строках где был ответ. 
М-шаг: новые полученные на Е-шаге таргеты используем для обучения модели. 
Движемся итеративно

In [27]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [37]:
num_epoch = 10
y_cur = y_train.copy()
mask = y_train.values == 0
model = LinearRegression()

for epoch in range(num_epoch):
    print('epoch:', epoch)

    model.fit(X_train, y_cur)
    y_pred = sigmoid(model.predict(X_train))
    np.putmask(y_pred, mask, 0)
    y_cur = y_pred

    rating_stat_model = {}
    all_train_players = len(train_dataset.player.unique())
    coef = model.coef_
    columns = X_train.columns
    for i in range(all_train_players):
      id_player = columns[i][7:]
      rating_stat_model[int(id_player)] = coef[i]
    players_df['rating_model'] = players_df.id.apply(lambda x: rating_stat_model[x] if x in rating_stat_model.keys() else 0)

    spearmanrmean_calc, kendalltau_calc = val_correlations(tournaments, 'rating_model')
    print('Корреляция Спирмена:', spearmanrmean_calc)
    print('Корреляция Кендалла:', kendalltau_calc)


epoch: 0


100%|██████████| 173/173 [08:48<00:00,  3.05s/it]


Корреляция Спирмена: 0.7278998268005902
Корреляция Кендалла: 0.5692668643841021
epoch: 1


100%|██████████| 173/173 [08:48<00:00,  3.06s/it]


Корреляция Спирмена: 0.7253963523714836
Корреляция Кендалла: 0.5665579248472311
epoch: 2


...

Корреляция Спирмена: 0.7260228211570547
Корреляция Кендалла: 0.5672369569983956
epoch: 8


100%|██████████| 173/173 [09:00<00:00,  3.12s/it]


Корреляция Спирмена: 0.7260219067098679
Корреляция Кендалла: 0.5672364390453177
epoch: 9


100%|██████████| 173/173 [09:09<00:00,  3.18s/it]

Корреляция Спирмена: 0.7260222579087936
Корреляция Кендалла: 0.5672368496604716


качество растет очень медленно

In [17]:
df_agg_players = train_dataset.groupby('player').answer.sum().reset_index().copy()
players_df.merge(df_agg_players, left_on='id', right_on='player').sort_values('rating_stat', ascending=False).head(10)


,id,name,patronymic,surname,rating_stat,rating_model,player,answer
5188,36844,Павел,Константинович,Щербина,0.972222,0.611356,36844,35
15357,133504,София,Евгеньевна,Лебедева,0.944444,0.548266,133504,32
42308,202410,Валентина,NaN,Подюкова,0.916667,0.576024,202410,33
28805,186002,Инга,Андрисовна,Лоренц,0.902778,0.576024,186002,33
54472,215496,Наталья,NaN,Артемьева,0.902778,0.548266,215496,32
54473,215497,Екатерина,NaN,Горелова,0.902778,0.548266,215497,32
54471,215495,Юлия,NaN,Крюкова,0.902778,0.548266,215495,32
32188,191332,Марина,Юрьевна,Савушкина,0.902778,0.520508,191332,31
23407,170977,Давид,Сергеевич,Кан,0.888889,0.548266,170977,32
11145,103161,Надежда,Фёдоровна,Бирюкова,0.888889,0.548266,103161,32


In [18]:
players_df.merge(df_agg_players, left_on='id', right_on='player').sort_values('rating_model', ascending=False).head(10)


,id,name,patronymic,surname,rating_stat,rating_model,player,answer
5188,36844,Павел,Константинович,Щербина,0.972222,0.611356,36844,35
28805,186002,Инга,Андрисовна,Лоренц,0.902778,0.576024,186002,33
42308,202410,Валентина,NaN,Подюкова,0.916667,0.576024,202410,33
40112,199963,Елена,Борисовна,Бровченко,0.875000,0.548266,199963,32
23407,170977,Давид,Сергеевич,Кан,0.888889,0.548266,170977,32
54473,215497,Екатерина,NaN,Горелова,0.902778,0.548266,215497,32
54471,215495,Юлия,NaN,Крюкова,0.902778,0.548266,215495,32
23570,171845,Михаил,Владимирович,Завьялов,0.888889,0.548266,171845,32
54472,215496,Наталья,NaN,Артемьева,0.902778,0.548266,215496,32
11145,103161,Надежда,Фёдоровна,Бирюкова,0.888889,0.548266,103161,32


в топ попадают участники которые сыграли скорее всего одну игру

# смотрим “рейтинг-лист” турниров по сложности вопросов

In [30]:

train_dataset.groupby('tournament_id').qty.mean().reset_index().merge(tournaments_df, left_on='tournament_id', right_on='id').sort_values('qty', ascending=False).head(10)


,tournament_id,qty,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
635,6090,14.960360,6090,Дзержинский марафон,2019-12-21T15:30:00+03:00,2019-12-22T15:30:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7551, 'name': 'Алексей', 'patronymic':...",None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ..."
123,5405,12.000000,5405,Кавалькада волхвов,2019-01-05T06:00:00+03:00,2019-01-05T16:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 9801, 'name': 'Егор', 'patronymic': 'А...",None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ..."
387,5709,8.032967,5709,Высшая лига ЧТ,2019-02-15T20:00:00+03:00,2019-04-30T19:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 9801, 'name': 'Егор', 'patronymic': 'А...",None,"{'1': 12, '2': 12, '3': 12, '4': 12, '5': 12, ..."
576,5976,8.000000,5976,Открытый Студенческий чемпионат Краснодарского...,2019-10-26T00:00:00+03:00,2020-03-01T14:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 40234, 'name': 'Александр', 'patronymi...",None,"{'1': 12, '2': 12, '3': 12, '4': 12, '5': 12, ..."
266,5564,8.000000,5564,Молодёжный чемпионат Нижегородской области,2019-02-01T00:00:00+03:00,2019-03-03T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 32901, 'name': 'Наиль', 'patronymic': ...",None,"{'1': 12, '2': 12, '3': 12, '4': 12, '5': 12, ..."
289,5592,8.000000,5592,Студенческая лига ЧТ,2019-02-16T00:00:00+03:00,2019-04-27T14:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 9801, 'name': 'Егор', 'patronymic': 'А...",None,"{'1': 12, '2': 12, '3': 12, '4': 12, '5': 12, ..."
350,5660,8.000000,5660,Первая лига ЧТ,2019-02-16T00:00:00+03:00,2019-04-27T14:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 9801, 'name': 'Егор', 'patronymic': 'А...",None,"{'1': 12, '2': 12, '3': 12, '4': 12, '5': 12, ..."
664,6150,6.500000,6150,Чемпионат Санкт-Петербурга. Высшая лига,2019-10-13T00:00:00+03:00,2019-12-01T15:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 26469, 'name': 'Алексей', 'patronymic'...",None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ..."
663,6149,6.500000,6149,Чемпионат Санкт-Петербурга. Первая лига,2019-10-13T00:00:00+03:00,2019-12-01T15:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 26469, 'name': 'Алексей', 'patronymic'...",None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ..."
346,5656,5.500000,5656,БЛИК,2019-06-01T14:00:00+03:00,2019-06-02T14:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 53126, 'name': 'Василий', 'patronymic'...",None,"{'1': 9, '2': 9, '3': 9, '4': 9, '5': 9, '6': ..."


В топ рейтинг-листа турниров по сложности попали 
Высшая лига ЧТ, Чемпионат Санкт-Петербурга.Высшая лига , Первая лига и тд. Что в целом отвечает логике 